In [1]:
import torch
print(torch.__version__)
print(f"CUDA: {torch.cuda.is_available()}")
# GPUの情報を確認
!nvidia-smi

2.7.1+cu128
CUDA: True
Mon Jul  7 20:17:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.64.01              Driver Version: 576.80         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        On  |   00000000:01:00.0  On |                  N/A |
|  0%   44C    P8             41W /  575W |    2593MiB /  32607MiB |      9%      Default |
|                                         |                        |                  N/A |
+------------------------

In [2]:
!python --version

Python 3.11.12


In [4]:
# ローカルLLMの例　（HuggingFaceのアカウントが必要です）

# --- モデルとトークナイザのロード (4-bit量子化設定) ---
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# モデル名
model_name = "google/gemma-2-2b-jpn-it"
# 4-bit量子化のための設定を作成
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", # 量子化のタイプ (nf4が推奨されることが多い)
    bnb_4bit_compute_dtype=torch.bfloat16, # 計算時のデータ型 (bfloat16が効率的)
    bnb_4bit_use_double_quant=True, # 二重量子化を有効にしてメモリをさらに節約
)

print("\nモデルとトークナイザのロードを開始します (4-bit量子化)...")
# トークナイザのロード
tokenizer = AutoTokenizer.from_pretrained(model_name)
# モデルのロード(RTX5090で300W)
# quantization_config パラメータに上記の設定を渡すことで、モデルが4-bitでロードされます。
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto", # モデルを自動でGPUに配置
)

print("\nモデルとトークナイザのロードが完了しました。")



モデルとトークナイザのロードを開始します (4-bit量子化)...


tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]


モデルとトークナイザのロードが完了しました。


In [16]:
# テキスト生成パイプラインの準備 ---
# Hugging Faceのpipelineを使うと簡単にテキスト生成ができます。
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

print("\nテキスト生成パイプラインの準備が完了しました。")

long_text = """
深層学習（ディープラーニング）とは何か？

深層学習（Deep Learning）は、機械学習（Machine Learning）の一種であり、人工知能（AI）の分野において、
現在最も広く活用され、注目を浴びている手法です。
機械学習は、「データから規則性やパターンを学習し、新しいデータに対して適切な予測や判断を行う」ための技術を指します。
その中でも、深層学習は、特に人間の脳神経細胞（ニューロン）の働きを数理モデル化した「ニューラルネットワーク」を、多層化したものです。
ニューラルネットワークの層を深く（deep）積み重ねることで、データに含まれる複雑なパターンや特徴を効果的に抽出・学習できることから、この名前がついています。

深層学習の仕組み（ニューラルネットワークの概要）

深層学習の根幹となるニューラルネットワークは、人工的に作られた神経細胞（ニューロン）の層が複数積み重なった構造をしています。
基本構成は以下の通りです。
入力層 (Input Layer)
データをニューラルネットワークに取り込む最初の層。画像、音声、テキストなどの生データがここに入ります。
中間層（隠れ層：Hidden Layers）
入力データから複雑な特徴を抽出し、それを学習する層。深層学習では、この中間層が複数存在することで、「より複雑な表現」を学習できます。
出力層 (Output Layer)
最終的な結果（分類、回帰予測、生成された画像など）を出力する層。
学習時、各ニューロンは重み（weights）とバイアス（biases）というパラメータを持ちます。
データが入力されると、このパラメータを使って各ニューロンが計算を行い、活性化関数（activation function）を通して出力を生成します。
この出力が次の層に伝播（forward propagation）されます。最終的にネットワークが出力した結果と正解データを比較し、誤差を逆向きに伝播
（back propagation）させてパラメータを調整します。これを「誤差逆伝播法（Back Propagation）」と呼びます。

深層学習が注目される理由

従来の機械学習では、専門家が特徴量を手作業で抽出（特徴エンジニアリング）する必要がありました。
例えば、画像認識なら「色や形状を特徴として選ぶ」などです。
しかし、深層学習は、データから特徴量を自動で抽出する能力を持ち、手作業の特徴設計が不要になります。
これを「表現学習（Representation Learning）」と言います。

深層学習が発展した背景には、以下の3つの要素が大きく寄与しています。
計算資源の向上（GPU、TPUなどの高速な並列処理ハードウェア）
大量かつ高品質なデータセットの整備（画像ならImageNet、テキストならWikipediaなど）
アルゴリズムの改善（Dropout、Batch Normalization、Transformerなど新たなアーキテクチャや技術の開発）
これらが揃ったことで、深層学習は2010年代以降に急速に発展を遂げました。
"""

# --- ステップ5: テキスト生成の実行 ---
# gemma-2-itモデルは、特定の形式のプロンプトで性能を発揮します。
prompt_template = "<start_of_turn>user\n{instruction}<end_of_turn>\n<start_of_turn>model\n"

length_tbl = [50,100,200,300,500,1000]

for length in length_tbl:
  instruction = f"以下の文を小学5年生でもわかるように、{length}文字程度に要約して文章にしなさい。箇条書きは禁止です。"
  prompt = prompt_template.format(instruction=instruction + f"\n\n{long_text}")

  print("\n--- instruction ---")
  print(instruction)

  # テキスト生成の実行 RTX5090で330W
  outputs = pipe(
      prompt,
      max_new_tokens=1024,
      do_sample=True,
      temperature=0.7,
      top_p=0.95,
  )

  print("\n--- モデルの応答 ---")
  # pipelineの出力はリスト形式で、生成されたテキストは 'generated_text' キーに格納されています
  generated_text = outputs[0]['generated_text']

  # プロンプト部分を除去して、モデルの応答だけを表示
  model_response = generated_text[len(prompt):].strip()
  print(model_response)
  print(f"{len(model_response)} 文字")

print("\n--- 処理完了 ---")



Device set to use cuda:0



テキスト生成パイプラインの準備が完了しました。

--- instruction ---
以下の文を小学5年生でもわかるように、50文字程度に要約して文章にしなさい。箇条書きは禁止です。

--- モデルの応答 ---
深層学習は、AIで複雑なパターンを学習する技術です。人工的な脳の仕組みで、新しいデータから良い予測ができるのです。
57 文字

--- instruction ---
以下の文を小学5年生でもわかるように、100文字程度に要約して文章にしなさい。箇条書きは禁止です。

--- モデルの応答 ---
深層学習は、人工知能（AI）で、人間の脳の働きを参考に、複雑なデータから、新しい情報や予測を学習する技術です。 
機械学習の一種で、入力層から複雑な中間層を経て、出力層で結果が出されます。
深層学習は、従来の機械学習では必要だった手作業の抽出を自動でできるため、注目されています。
140 文字

--- instruction ---
以下の文を小学5年生でもわかるように、200文字程度に要約して文章にしなさい。箇条書きは禁止です。

--- モデルの応答 ---
深層学習とは、人工知能の分野で、人間の脳の仕組みを参考に、複雑なパターンを学習する技術です。

深層学習は、たくさんの層を重ねた「ニューラルネットワーク」を使って、データから複雑な特徴を自動で抽出します。 
ニューラルネットワークは、入力層、中間層、出力層から構成され、それぞれの層は複雑なパターンを学習します。

深層学習は、従来の機械学習では必要だった専門家の特徴量設計を自動化できる点が注目されています。
これは、大量の画像やテキストデータから自動的に特徴を抽出できるため、複雑なパターンを学習できます。
255 文字

--- instruction ---
以下の文を小学5年生でもわかるように、300文字程度に要約して文章にしなさい。箇条書きは禁止です。

--- モデルの応答 ---
深層学習とは、人工知能（AI）で、人間が脳のように複雑なパターンを学習する技術です。

機械学習と違い、深層学習は「ニューラルネットワーク」と呼ばれる構造で、複雑なデータから、人間が学習するような表現を自動で学習します。

ニューラルネットワークは、入力層から、中間層、そして出力層で構成されます。

深層

In [17]:
# --- GPUメモリの解放 ---
import gc
del model
del tokenizer
del pipe
gc.collect()
torch.cuda.empty_cache()
print("\nGPUメモリを解放しました。")


GPUメモリを解放しました。
